Before attempting to load data, make sure to standardize your excel sheet to make your life easier.
The excel sheet should have:
-Consistent category/column labels across subjects
-First row is category/column labels
-Keep extra information elsewhere
-Be 'snug' left and top (we want it to be a simple table starting with corner cell)

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy.io
import seaborn as sns
from scipy.signal import butter, hilbert, filtfilt
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import datetime
import os

In [28]:
def get_mood_data(file_name):
    """
    Evaluates an excel spreadsheet for IMS metrics and timestamps. Returns
    a list of the depression and anxiety scores and a list of times.

    Inputs:
    file_name - a standardized excel sheet with category labels in the first row.
    """
    df = pd.read_excel(file_name)
    df = df[['time', 'depression score', 'anxiety score']].dropna()
    dep_score = [int(i) for i in df['depression score']]
    anx_score = [int(i) for i in df['anxiety score']]
    return list(dep_score), list(anx_score), list(df['time'])

def get_neural(neural_data):
    """
    Loads in the neural dataset and returns an array of the samples, an array of time,
    and sampling frequency.

    Inputs:
    neural_data: h5 file containing neural data
    """
    # Load neural data
    fin = h5py.File(neural_data, 'r')

    # Express channels
    chan_label = fin.get('chanLabels')[()]

    if isinstance(chan_label, bytes):
        chan_clean = chan_label.decode('utf-8').replace("'",'').replace(',','').replace('[','').replace(']','')
        chan_label = chan_clean.split()

    # Evaluate time
    t_start = fin.get('start_timestamp')[()]
    t_start_utc = datetime.datetime.utcfromtimestamp(int(t_start))

    # Extract neural data
    data_ecog = fin.get('dataset')
    fs = int(fin.get('f_sample')[()])
    t = np.linspace(0, len(data_ecog[0]) / fs, len(data_ecog[0]))
    total_time = len(data_ecog[0]) / fs
    t_end_utc = t_start_utc + datetime.timedelta(seconds = total_time)
    return data_ecog, chan_label, t, t_start_utc, t_end_utc, fs


def get_windows(neural_data, ims_file_name, window_length):
    """
    Splices an h5 dataset into windows of neural activity corresponding to
    VAS measures. Returns None if no IMS records within neural recording.

    Inputs:
    neural_data - h5 file containing neural data and labels
    ims_file_name - standardized excel sheet containing ims information
    window_length - length of extraction windows in minutes
    """
    # Load vas data
    dep_score, anx_score, ims_timestamps = get_mood_data(ims_file_name)

    # Load neural data
    data_mood, chs, t, t_start, t_end, fs = get_neural(neural_data)

    # Establish time buffers for adequate samples
    t_end_buffer = t_end - datetime.timedelta(seconds=60*window_length/2) # ensure enough samples
    t_start_buffer = t_start + datetime.timedelta(seconds=60*window_length/2)
    print('t_start_buffer', t_start_buffer)
    print('t_end_buffer', t_end_buffer)

    # Calculate timestamps and target sample
    ims_times = [x.to_pydatetime().timestamp() - t_start.timestamp() for x in ims_timestamps if t_start_buffer < x < t_end_buffer]
    if len(ims_times) == 0:
        return None, None, None, None, None, None
    dep_score_tracker = [dep_score[i] for i in range(len(dep_score)) if t_start_buffer < ims_timestamps[i] < t_end_buffer]
    anx_score_tracker = [anx_score[i] for i in range(len(anx_score)) if t_start_buffer < ims_timestamps[i] < t_end_buffer]
    target_sample = [int(i) * fs for i in ims_times]
    print('target_sample', target_sample)
    print('len channels', len(chs))

    # Create windows
    window_length_s = window_length*60*fs
    start_frame = [int(i - window_length_s/2) for i in target_sample]
    stop_frame = [int(i + window_length_s/2) for i in target_sample]

    # Extract windows
    neural_windows = np.zeros((len(target_sample), len(chs), stop_frame[0] - start_frame[0]))
    file_tracker = []
    print('neural_windows shape', neural_windows.shape)
    print('data_mood shape', data_mood.shape)
    
    for i in range(len(target_sample)):
        neural_windows[i, :, :] = data_mood[:, start_frame[i]:stop_frame[i]]
        file_tracker.append(neural_data)
    
    return neural_windows, dep_score_tracker, anx_score_tracker, fs, chs, file_tracker, target_sample


def extract_windows(directory, ims_file_name, window_length, output_name):
    """
    Joins multiple windows of neural data together, maintaining
    associated labels.
    
    Inputs:
    directory - a path to the directory containing eeg data
    ims_file_name - an excel spreadsheet of IMS records
    window_length - length of extraction windows in minutes
    output_name - resultant h5 file
    """

    # Initialize combined arrays / list
    combined_windows = None
    combined_target_sample = None
    combined_dep_score = None
    combined_anx_score = None
    combined_file_tracker = []

    # Sort neural data chronologically
    sorted_dir = sorted(os.listdir(directory))
    if '.DS_Store' in sorted_dir:
        sorted_dir.remove('.DS_Store')
    
    for file in sorted_dir:
        neural_windows, dep_score_tracker, anx_score_tracker, fs, chs, file_tracker, target_sample = get_windows(directory + file, 
                                                                               ims_file_name,
                                                                               window_length)
        # Skip over files without IMS records
        if dep_score_tracker is None:
            continue

        # Evaluate, starting with first dataset containing VAS records
        print('Extracting ' + str(len(dep_score_tracker)) + ' IMS from ' + file)
        if len(combined_file_tracker) == 0:
            combined_windows = neural_windows
            combined_target_sample = target_sample
            combined_dep_score = dep_score_tracker
            combined_anx_score = anx_score_tracker
            combined_file_tracker = file_tracker
            fsample = fs
            channels = chs
        else:
            combined_windows = np.concatenate([combined_windows, neural_windows])
            combined_file_tracker = combined_file_tracker + file_tracker
            combined_target_sample = np.concatenate([combined_target_sample, target_sample])
            combined_dep_score = np.concatenate([combined_dep_score, dep_score_tracker])
            combined_anx_score = np.concatenate([combined_anx_score, anx_score_tracker])

    # Create h5 file
    ims_out = h5py.File(output_name, "w")
    ims_out.create_dataset("neural_windows", data=combined_windows, chunks=True)
    ims_out.create_dataset("depression scores", data=combined_dep_score)
    ims_out.create_dataset("anxiety scores", data=combined_anx_score)
    ims_out.create_dataset("fs", data=fsample, dtype='i8')
    ims_out.create_dataset("file", data=combined_file_tracker)
    dt = h5py.special_dtype(vlen=str)
    ims_out.create_dataset("channels", data=channels, dtype=dt)
    ims_out.create_dataset("target_sample", data=combined_target_sample, dtype = 'i')
    print('Extraction Complete')
    return


In [ ]:
#Main

directory = '/Users/haleh/Documents/School/UW/Research/Jeffery Herron/Mood Variation Decoding/Neural Data/eb06df40 Neural/' #specific subject and day name
output = 'IMS_eb06df40_6_20min.h5'

extract_windows(directory=directory, ims_file_name='survey_responses_ims.xlsx', window_length=20, output_name=output)

##NOTE: for some reason, there are 1268 channels, this sounds wrong, need to debug furhter

In [ ]:
#Filtering and calculating power in band
def get_PIB(data,fs,f_low,f_high,order,axis):
    '''
    This function computes the average power in frequency band for any signal
    Input
        data = any signal in the form of (time, voltage)
        f_low = low cutoff point of the desired frequency range
        f_high = high cutoff point of the desired frequency range
        fs = sampling frequency
        order = order of butterworth bandpass filter
        axis = axis to compute across (0 = 1D  signals, 1 = 2D signals)

    Outputs
        average = average power across frequency band
  '''
    #Butterworth bandpass filter
    b,a = butter(order,[f_low,f_high],fs=fs,btype='band') 
    #Filtering the input signal with the butterworth bandpass filter
    filtered = filtfilt(b,a,data)
    #Hilbert transform of the filtered signal
    data_hilbert = hilbert(filtered)
    #power of the transformed signal
    power = (abs(data_hilbert))**2
    #Sum of the power across the time axis of the signal
    total = np.sum(power, axis=axis)
    return total

def predict_pain(data_epoch, fsample, dep_score, ch_select, f_bands, f_band, label_splitter, test_split=0.30):
    PIB_feat = np.zeros((len(dep_score), np.shape(data_epoch)[2]))
    for vas in range(len(dep_score)):
        for ep in range(np.shape(data_epoch)[2]):
            low = f_bands[f_band][0]
            high = f_bands[f_band][1]
            PIB_feat[vas, ep] = get_PIB(signal=data_epoch[vas, ch_select, ep, :],
                                        fs=fsample, f_low=low, f_high=high,
                                        order=3, axis=0)
    labels = np.repeat(dep_score, np.shape(data_epoch)[2])
    split_labels = labels <= label_splitter

    # Standardize features
    PIB_feat = np.ravel(PIB_feat)
    PIB_mean = np.mean(PIB_feat)
    PIB_std = np.std(PIB_feat)
    if PIB_std == 0:
      return 0
    PIB_feat_st = (PIB_feat - PIB_mean) / (PIB_std)
    PIBs = np.reshape(np.array(PIB_feat_st), (len(PIB_feat_st), 1))

    # Apply logistic regression
    logr = LogisticRegression(max_iter=10000)
    X_train, X_test, y_train, y_test = train_test_split(PIBs, split_labels, test_size=test_split)
    logr.fit(X_train, y_train)
     
    return logr.score(X_test, y_test)

In [ ]:
# Main

if __name__ == '__main__':

    # Load in the dataset
    fin = h5py.File('IMS_eb06df40_6_20min.h5', 'r') # Extracted windows
    pain_data = fin.get('neural_windows')[()]
    chans = fin.get('channels')[()]
    dep_scores = fin.get('depression scores')
    anx_scores = fin.get('anxiety scores')
    fs = fin.get('fs')[()]
    print(dep_scores)

    # Calculate features
    f_bands = [[0.1,4],[4,8],[8,12],[12,30],[30,55],[65,115]] # Establish neural frequency band indices 
    f_bands_label = ["delta", "theta", "alpha", "beta", "gamma", "high gamma"]

    # Epoch signal
    epochs = 5
    data_epoched = pain_data.reshape((len(dep_scores), len(chans), epochs, -1))

    # Get features and labels
    pred = predict_pain(data_epoch=data_epoched, fsample=fs, dep_score=dep_scores,
                       f_bands=f_bands, ch_select=40, f_band=4, label_splitter=5)
    # # # Evaluate combinations
    acc_matrix = np.zeros((len(chans), len(f_bands)))

    for i in range(1, len(chans)):
        for j in range(len(f_bands)):
            acc_matrix[i, j] = predict_pain(data_epoch=data_epoched, fsample=fs, intensity=dep_scores,
                       f_bands=f_bands, ch_select=i, f_band=j, label_splitter=4)